# Solving the furnace loading problem using Graph Theory and LPP solver 

In [1]:
# import modules
from pulp import * # For LPP solver
import pandas as pd 
import numpy as np
import os# Changing the directory to where the CSV files ares stored
os.chdir(r'''/Users/murali/Desktop''')

# Data Collection from Parts Dataset

In [ ]:
data = pd.read_excel('.xlsx')
data.head()
e = data['ei(mm)']
n = data['ni(mm)']
quantity = data['Order_qty']
N = e.shape[0]
parts = list(range(1, len(quantity)+1))

# Solving 2-D LPP

In [ ]:
# Defining the nature of the problem
nesting_problem = LpProblem("Parts Nesting problem: ", LpMaximize)

# Decision variables

# 1. No of inner parts k to be nested in outer parts i
combinations = []
for i in range(1,N+1):
    for j in range(1,N+1):
        combinations +=[(i,j)] 
variables = LpVariable.dicts('s',combinations, lowBound = 0,cat = LpInteger)

# 2. Define Pik = profit of nesting inner part k to be nested in outer parts i 

p = np.zeros((len(combinations), 1)).reshape(N,N)

for i in range(N):
    for k in range(N):
        if (n[k] - e[i] >= 50):
            p[i,k] = e[i]/n[k]

nesting_problem += lpSum([variables[i+1, j+1] * p[j,i] for i in range(N) for j in range(N)])

for i in range(N):
    nesting_problem += lpSum([variables[i+1,k+1] for k in range(N)]) <= quantity[i]

for k in range(N):
    nesting_problem += lpSum([variables[i+1,k+1] for i in range(N)]) <= quantity[k]

nesting_problem.solve()
print("Status:", LpStatus[nesting_problem.status])

nest_solution = []

for v in nesting_problem.variables():
    nest_solution += [[v.varValue]]
    print (v.name, "=", v.varValue)
nest_solution = np.asarray(nest_solution).reshape(N,N).T

# The optimised objective function value is printed to the screen
print ("Total Cost of Ingredients per can = ", value(nesting_problem.objective))


# Deep First Search(DFS) Function 

In [ ]:
def dfs(graph, start):
    visited, stack = set(), [start]
    while stack:
        vertex = stack.pop()
        if vertex not in visited:
            visited.add(vertex)
            stack.extend(graph[vertex] - visited)
    return visited

# Sorting Function

In [ ]:
def sort_nest(lists):
    l = len(lists)
    for i in range(l):
        for j in range(i,l):
            if len(lists[i]) < len(lists[j]):
                temp = lists[i]
                lists[i] = lists[j] 
                lists[j] = temp
    return lists
                

# Graph Theory

In [ ]:
# Using NetworkX package
import networkx as nx
G = nx.DiGraph()
nodes = []
edges =[]
for i in range(N):
    for j in range(N):
        if nest_solution[i,j] != 0:
            nodes += [str(i+1)]
            nodes += [str(j+1)]
            edges += [(str(i+1), str(j+1))]
            
G.add_edges_from(edges, weight = -1)
nodes = set(nodes)

nest_level = []

type_nest = []


for i in nodes:
    for j in nodes:
        type_nest += nx.all_simple_paths(G, source = str(int(i)),target = str(int(j)))

# Now sorting the combinations interms of length of the multi-level nesting 
sort_type_nest = sort_nest(type_nest)

In [ ]:
sort_type_nest

# Final_Answer : "Parts Nesting Problem"

In [3]:
nest_solution

NameError: name 'nest_solution' is not defined

In [27]:
quantities = nest_solution.copy()
answer =[]
for pattern in sort_type_nest:
    if len(pattern) != 1:
        sub_patterns = []
        for j in range(len(pattern)-1):
            sub_patterns += [(str(pattern[j]), str(pattern[j+1]))]
        # Accessing each edge
        quant =[]
        for k in sub_patterns:
            quant.append(int(quantities[int(k[0])-1, int(k[1])-1]))
        number = min(quant)
        answer += [number]
        print('No of parts with the combination', pattern, 'is', number)
        
        # Update the list now
        for k in sub_patterns:
            b = int(k[1])-1 # a and b are indices of the nest_solution matrix
            a = int(k[0])-1
            if quantities[a,b] != 0:
                quantities[a,b] = quantities[a,b] - number
                
                

# Now single parts which are remained after nesting
used_parts = []
for i in sorted(nodes):
    count = 0
    loop = 0
    for k in sort_type_nest:
        if i in k:
            count += answer[loop]
        loop += 1
    used_parts += [count]

used_parts_dict = dict(zip(sorted(nodes), used_parts)) # here keys are string integers
list_of_keys = list(range(1, len(quantity)+1))
all_parts_quantity = dict(zip(list_of_keys,quantity)) # here keys are integers

left_over_quantity = {}
for i in sorted(nodes):
    a =(all_parts_quantity[int(i)] - used_parts_dict[i])
    left_over_quantity.update( {i : a} )
for i in parts:
    for j in nodes:
        if str(i) == j:
            print("Quantity of part",i,"to be placed alone after removing the nestings is",left_over_quantity[j])

# Not nested at all
# Note that nestings are only one level nestings
# Now making sets
no_nested = 0
no_nested_parts = []
for i in range(N):
    if sum(nest_solution[i,:]) == 0:
        if sum(nest_solution[:,i]) == 0:
            print("Quantity of part",i+1,"to be placed alone after removing the nestings is",all_parts_quantity[i+1])
            no_nested += 1 
            print('No of no_nested is', no_nested)
        


No of parts with the combination ['1', '2', '4'] is 20
No of parts with the combination ['1', '3', '4'] is 80
No of parts with the combination ['2', '4'] is 0
No of parts with the combination ['1', '2'] is 62
No of parts with the combination ['1', '3'] is 0
No of parts with the combination ['3', '4'] is 0
Quantity of part 1 to be placed alone after removing the nestings is 78
Quantity of part 2 to be placed alone after removing the nestings is 0
Quantity of part 3 to be placed alone after removing the nestings is 0
Quantity of part 4 to be placed alone after removing the nestings is 0


In [28]:
list_of_keys

[1, 2, 3, 4]

In [29]:
nest_solution
# Assume part2
sum(nest_solution[:,1]) + sum(nest_solution[1,:]) - quantity[1]

20.0

# Layer Loading Problem

In [8]:
data2 = pd.read_excel('layer_loading_parts.xlsx')
data2.head()

,Set,order_qty,ek(mm),ni(mm),height(mm)
0,1,1,325.2,293.7,107.2
1,2,1,325.2,293.7,107.2
2,3,1,325.2,293.7,107.2
3,4,1,216.8,184.1,107.2
4,5,1,216.8,184.1,107.2


In [9]:
e2 = data2['ek(mm)']
n2 = data2['ni(mm)']
quantity2 = data2['order_qty']
N2 = np.sum(quantity2)
parts2 = list(range(1, len(quantity2)+1))
print("These are part numbers should be assigned:", parts2)
print("Total number of parts are:", N2)
# Descending order(part 1 is higher in dimension and part 4 is lowest)

These are part numbers should be assigned: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
Total number of parts are: 19


# Solving layer loading LPP

In [10]:
# Nature of the problem
layer_loading = LpProblem("Layer Loading problem:", LpMinimize)

# Decision variables for layer loading problem
# 1. Coordinate decision variables
x_vars = LpVariable.dicts('x',parts2, lowBound = 0, cat = LpContinuous)
y_vars = LpVariable.dicts('y',parts2, lowBound = 0, cat = LpContinuous)

F = 1 # Here only one layer is used later on this need to be modified
L = 980
W = 651

# 2. Position either left, top, bottom and right variables
combinations_2 = []
for i in range(1,N2+1):
    for k in range(1,N2+1):
        if i != k:
            combinations_2 +=[(i,k)]
a_vars = LpVariable.dicts('a',combinations_2, cat = LpBinary)
b_vars = LpVariable.dicts('b',combinations_2, cat = LpBinary)
c_vars = LpVariable.dicts('c',combinations_2, cat = LpBinary)
d_vars = LpVariable.dicts('d',combinations_2, cat = LpBinary)

# Now adding objective function which is function of decision variables
layer_loading += 1, "Arbitrary objective function"

# Now adding layer dimension constraints
for i in range(N2):
    layer_loading += x_vars[i+1] <= L
    layer_loading += x_vars[i+1] >= e2[i]
    layer_loading += y_vars[i+1] <= W
    layer_loading += y_vars[i+1] >= e2[i]

# Now adding the constraints which take care of overlapping
M = 100000000 # very large number 1million is my assumption
for i in range(N2):
    for k in range(N2):
        if i != k:
            layer_loading += x_vars[k+1] - e2[k] + M*(1 - a_vars[(i+1,k+1)]) >= x_vars[i+1]
            layer_loading += x_vars[i+1] - e2[i] + M*(1 - b_vars[(i+1,k+1)]) >= x_vars[k+1]
            layer_loading += y_vars[k+1] - e2[k] + M*(1 - c_vars[(i+1,k+1)]) >= y_vars[i+1]
            layer_loading += y_vars[i+1] - e2[i] + M*(1 - d_vars[(i+1,k+1)]) >= y_vars[k+1]
            layer_loading += a_vars[(i+1,k+1)] + b_vars[(i+1,k+1)] + c_vars[(i+1,k+1)] + d_vars[(i+1,k+1)] >= 1


In [11]:
layer_loading

Layer Loading problem::
MINIMIZE
1
SUBJECT TO
_C1: x_1 <= 980

_C2: x_1 >= 325.2

_C3: y_1 <= 651

_C4: y_1 >= 325.2

_C5: x_2 <= 980

_C6: x_2 >= 325.2

_C7: y_2 <= 651

_C8: y_2 >= 325.2

_C9: x_3 <= 980

_C10: x_3 >= 325.2

_C11: y_3 <= 651

_C12: y_3 >= 325.2

_C13: x_4 <= 980

_C14: x_4 >= 216.8

_C15: y_4 <= 651

_C16: y_4 >= 216.8

_C17: x_5 <= 980

_C18: x_5 >= 216.8

_C19: y_5 <= 651

_C20: y_5 >= 216.8

_C21: x_6 <= 980

_C22: x_6 >= 162.6

_C23: y_6 <= 651

_C24: y_6 >= 162.6

_C25: x_7 <= 980

_C26: x_7 >= 162.6

_C27: y_7 <= 651

_C28: y_7 >= 162.6

_C29: x_8 <= 980

_C30: x_8 >= 162.6

_C31: y_8 <= 651

_C32: y_8 >= 162.6

_C33: x_9 <= 980

_C34: x_9 >= 162.6

_C35: y_9 <= 651

_C36: y_9 >= 162.6

_C37: x_10 <= 980

_C38: x_10 >= 108.4

_C39: y_10 <= 651

_C40: y_10 >= 108.4

_C41: x_11 <= 980

_C42: x_11 >= 108.4

_C43: y_11 <= 651

_C44: y_11 >= 108.4

_C45: x_12 <= 980

_C46: x_12 >= 108.4

_C47: y_12 <= 651

_C48: y_12 >= 108.4

_C49: x_13 <= 980

_C50: x_13 >= 108.4


In [ ]:
layer_loading.solve(solver = GLPK_CMD())
print("Status:", LpStatus[layer_loading.status])

In [ ]:
pulp.pulpTestAll()

In [ ]:
layer_loading_solution = []

for v in layer_loading.variables():
    layer_loading_solution += [[v.varValue]]
    print (v.name, "=", v.varValue)

# The optimised objective function value is printed to the screen
print ("Total Cost of Ingredients per can = ", value(layer_loading.objective))